In [1]:
import numpy as np
import pandas as pd
import folium
import datetime as dt
import matplotlib.pyplot as plt
from geopy.distance import geodesic

In [2]:
names = ['time', 'lat', 'lon']
bus = pd.read_csv("GPSDataclean.csv",header=None, names=names,low_memory=False, usecols=[0,2,3]) #csvファイルから0,2,3行目をとってきて名前をつけてる
bus = bus.drop(bus.index[0]) #初期データのindexを削除
bus = bus.dropna() #空白を補完
bus = bus.reset_index(drop=True)
len(bus),print(bus.dtypes)

time    object
lat     object
lon     object
dtype: object


(296933, None)

In [3]:
print(bus[bus['lat'].str.contains(':', na=False)])
drop_index = bus.index[bus['lat'].str.contains(':', na=False)] #latに：などが入っているものを削除
bus = bus.drop(drop_index)

                   time      lat lon
105482  2022/1/26 19:25  17:10.3   0
134647   2022/2/17 6:57  17:10.5   0
156215   2022/2/5 16:13  00:24.4   0
211749   2022/4/12 9:22  17:09.7   0


In [4]:
bus = bus.astype({'lat': float, 'lon': float}) #緯度経度をfloat型に変更
bus['time'] = pd.to_datetime(bus['time']) #timeをdatetime型に変更
bus.dtypes

time    datetime64[ns]
lat            float64
lon            float64
dtype: object

In [5]:
bus = bus.drop(bus.index[(bus['lat']<40) | (bus['lat']>41)])
bus = bus.drop(bus.index[(bus['lon']<140) | (bus['lon']>141)]) #不必要なデータの削除

In [6]:
day_time=bus["time"].dt.time
day_time= pd.DataFrame({'day_time':day_time})
bus=pd.concat([bus,day_time],axis=1) #時間のみのデータを作り元データに加える

In [7]:
stops = pd.read_csv("oowani_stop.csv",usecols=[2,4,5,13,14,15,16,17,18,19,20,21,22,23,24,25,26]) #バス停のデータから緯度経度や時刻表のデータをとってくる
stops = stops.dropna(subset=['stop_lat', 'stop_lon'])
stops = stops.astype({'stop_lat': float, 'stop_lon': float})

In [8]:
route = pd.read_csv('route_oowani.csv') #大鰐線のルートを読み込む
route=route.drop(route.columns[0],axis=1)
dis_scale = 84.78234392637954 #緯度経度をｍに変更
route = route.values.tolist()
def xy2no(xy):
 distances=np.sqrt(np.sum((xy-route)**2,axis=1))*dis_scale #ルートからの距離を計算
 xy2r_max=np.amin(distances)
 if xy2r_max>0.1:
  no=-1 #100m以上離れいている点を-1とする
 else:  
   no=np.argmin(distances)
 return no
I=len(stops)
nos=[]
for i in range(0,I):
 nos.append(xy2no(np.array([stops.iloc[i,1],stops.iloc[i,2]])))
nos_df = pd.DataFrame({'nos':nos}) #nosを加える
stops = pd.concat([stops, nos_df], axis=1)

In [9]:
for i in range(7,18):
    exec_command='stops["Stop_'+str(i)+'"]=pd.to_datetime(stops["Stop_'+str(i)+'"])'
    exec(exec_command) #時間をdatetime型にする

In [10]:
day = pd.to_datetime('2023 8:00:00') #適当な日付を決める
for i in range (7,18):
    timedelta = pd.DataFrame(stops["Stop_8"] - day).rename(columns={"Stop_8":"Stop_day_8"})

stops = pd.concat([stops,timedelta],axis=1)
stops

,stop_name,stop_lat,stop_lon,Stop_7,Stop_8,Stop_9,Stop_10,Stop_11,Stop_12,Stop_13,Stop_14,Stop_15,Stop_16,Stop_17,Stop_18_1,Stop_18_2,Stop_19,nos,Stop_day_8
0,弘前ﾊﾞｽﾀｰﾐﾅﾙ前,40.600088,140.480911,2023-03-04 07:20:00,2023-03-04 08:00:00,2023-03-04 09:00:00,2023-03-04 10:00:00,2023-03-04 11:00:00,2023-03-04 12:00:00,2023-03-04 13:00:00,2023-03-04 14:00:00,2023-03-04 15:00:00,2023-03-04 16:00:00,2023-03-04 17:00:00,18:00:00,18:30:00,19:00:00,24,62 days 00:00:00
1,弘前駅前,40.598758,140.484247,2023-03-04 07:21:00,2023-03-04 08:01:00,2023-03-04 09:01:00,2023-03-04 10:01:00,2023-03-04 11:01:00,2023-03-04 12:01:00,2023-03-04 13:01:00,2023-03-04 14:01:00,2023-03-04 15:01:00,2023-03-04 16:01:00,2023-03-04 17:01:00,18:01:00,18:31:00,19:01:00,0,62 days 00:01:00
2,上代官町,40.598536,140.476086,2023-03-04 07:25:00,2023-03-04 08:05:00,2023-03-04 09:05:00,2023-03-04 10:05:00,2023-03-04 11:05:00,2023-03-04 12:05:00,2023-03-04 13:05:00,2023-03-04 14:05:00,2023-03-04 15:05:00,2023-03-04 16:05:00,2023-03-04 17:05:00,18:02:00,18:32:00,19:05:00,76,62 days 00:05:00
3,上土手町,40.596157,140.477692,2023-03-04 07:27:00,2023-03-04 08:07:00,2023-03-04 09:07:00,2023-03-04 10:07:00,2023-03-04 11:07:00,2023-03-04 12:07:00,2023-03-04 13:07:00,2023-03-04 14:07:00,2023-03-04 15:07:00,2023-03-04 16:07:00,2023-03-04 17:07:00,18:07:00,18:37:00,19:07:00,109,62 days 00:07:00
4,中央松森町,40.591484,140.481957,2023-03-04 07:29:00,2023-03-04 08:09:00,2023-03-04 09:09:00,2023-03-04 10:09:00,2023-03-04 11:09:00,2023-03-04 12:09:00,2023-03-04 13:09:00,2023-03-04 14:09:00,2023-03-04 15:09:00,2023-03-04 16:09:00,2023-03-04 17:09:00,18:09:00,18:39:00,19:09:00,173,62 days 00:09:00
5,松森町角,40.588628,140.484458,2023-03-04 07:29:00,2023-03-04 08:09:00,2023-03-04 09:09:00,2023-03-04 10:09:00,2023-03-04 11:09:00,2023-03-04 12:09:00,2023-03-04 13:09:00,2023-03-04 14:09:00,2023-03-04 15:09:00,2023-03-04 16:09:00,2023-03-04 17:09:00,18:09:00,18:39:00,19:09:00,211,62 days 00:09:00
6,取上,40.585583,140.487824,2023-03-04 07:30:00,2023-03-04 08:10:00,2023-03-04 09:10:00,2023-03-04 10:10:00,2023-03-04 11:10:00,2023-03-04 12:10:00,2023-03-04 13:10:00,2023-03-04 14:10:00,2023-03-04 15:10:00,2023-03-04 16:10:00,2023-03-04 17:10:00,18:10:00,18:40:00,19:10:00,256,62 days 00:10:00
7,大清水,40.581048,140.494099,2023-03-04 07:31:00,2023-03-04 08:11:00,2023-03-04 09:11:00,2023-03-04 10:11:00,2023-03-04 11:11:00,2023-03-04 12:11:00,2023-03-04 13:11:00,2023-03-04 14:11:00,2023-03-04 15:11:00,2023-03-04 16:11:00,2023-03-04 17:11:00,18:11:00,18:41:00,19:11:00,330,62 days 00:11:00
8,大清水局前,40.579968,140.496199,2023-03-04 07:32:00,2023-03-04 08:12:00,2023-03-04 09:12:00,2023-03-04 10:12:00,2023-03-04 11:12:00,2023-03-04 12:12:00,2023-03-04 13:12:00,2023-03-04 14:12:00,2023-03-04 15:12:00,2023-03-04 16:12:00,2023-03-04 17:12:00,18:12:00,18:42:00,19:12:00,351,62 days 00:12:00
9,門外十文字,40.579040,140.499638,2023-03-04 07:33:00,2023-03-04 08:13:00,2023-03-04 09:13:00,2023-03-04 10:13:00,2023-03-04 11:13:00,2023-03-04 12:13:00,2023-03-04 13:13:00,2023-03-04 14:13:00,2023-03-04 15:13:00,2023-03-04 16:13:00,2023-03-04 17:13:00,18:13:00,18:43:00,19:13:00,383,62 days 00:13:00


In [11]:
stops['Stop_18_1'] = pd.to_datetime(stops['Stop_18_1'])

In [13]:
stops['Stop_S_8']=stops['Stop_day_8'].dt.seconds #時間を秒に変換する
stops

,stop_name,stop_lat,stop_lon,Stop_7,Stop_8,Stop_9,Stop_10,Stop_11,Stop_12,Stop_13,Stop_14,Stop_15,Stop_16,Stop_17,Stop_18_1,Stop_18_2,Stop_19,nos,Stop_day_8,Stop_S_8
0,弘前ﾊﾞｽﾀｰﾐﾅﾙ前,40.600088,140.480911,2023-03-04 07:20:00,2023-03-04 08:00:00,2023-03-04 09:00:00,2023-03-04 10:00:00,2023-03-04 11:00:00,2023-03-04 12:00:00,2023-03-04 13:00:00,2023-03-04 14:00:00,2023-03-04 15:00:00,2023-03-04 16:00:00,2023-03-04 17:00:00,2023-03-04 18:00:00,18:30:00,19:00:00,24,62 days 00:00:00,0
1,弘前駅前,40.598758,140.484247,2023-03-04 07:21:00,2023-03-04 08:01:00,2023-03-04 09:01:00,2023-03-04 10:01:00,2023-03-04 11:01:00,2023-03-04 12:01:00,2023-03-04 13:01:00,2023-03-04 14:01:00,2023-03-04 15:01:00,2023-03-04 16:01:00,2023-03-04 17:01:00,2023-03-04 18:01:00,18:31:00,19:01:00,0,62 days 00:01:00,60
2,上代官町,40.598536,140.476086,2023-03-04 07:25:00,2023-03-04 08:05:00,2023-03-04 09:05:00,2023-03-04 10:05:00,2023-03-04 11:05:00,2023-03-04 12:05:00,2023-03-04 13:05:00,2023-03-04 14:05:00,2023-03-04 15:05:00,2023-03-04 16:05:00,2023-03-04 17:05:00,2023-03-04 18:02:00,18:32:00,19:05:00,76,62 days 00:05:00,300
3,上土手町,40.596157,140.477692,2023-03-04 07:27:00,2023-03-04 08:07:00,2023-03-04 09:07:00,2023-03-04 10:07:00,2023-03-04 11:07:00,2023-03-04 12:07:00,2023-03-04 13:07:00,2023-03-04 14:07:00,2023-03-04 15:07:00,2023-03-04 16:07:00,2023-03-04 17:07:00,2023-03-04 18:07:00,18:37:00,19:07:00,109,62 days 00:07:00,420
4,中央松森町,40.591484,140.481957,2023-03-04 07:29:00,2023-03-04 08:09:00,2023-03-04 09:09:00,2023-03-04 10:09:00,2023-03-04 11:09:00,2023-03-04 12:09:00,2023-03-04 13:09:00,2023-03-04 14:09:00,2023-03-04 15:09:00,2023-03-04 16:09:00,2023-03-04 17:09:00,2023-03-04 18:09:00,18:39:00,19:09:00,173,62 days 00:09:00,540
5,松森町角,40.588628,140.484458,2023-03-04 07:29:00,2023-03-04 08:09:00,2023-03-04 09:09:00,2023-03-04 10:09:00,2023-03-04 11:09:00,2023-03-04 12:09:00,2023-03-04 13:09:00,2023-03-04 14:09:00,2023-03-04 15:09:00,2023-03-04 16:09:00,2023-03-04 17:09:00,2023-03-04 18:09:00,18:39:00,19:09:00,211,62 days 00:09:00,540
6,取上,40.585583,140.487824,2023-03-04 07:30:00,2023-03-04 08:10:00,2023-03-04 09:10:00,2023-03-04 10:10:00,2023-03-04 11:10:00,2023-03-04 12:10:00,2023-03-04 13:10:00,2023-03-04 14:10:00,2023-03-04 15:10:00,2023-03-04 16:10:00,2023-03-04 17:10:00,2023-03-04 18:10:00,18:40:00,19:10:00,256,62 days 00:10:00,600
7,大清水,40.581048,140.494099,2023-03-04 07:31:00,2023-03-04 08:11:00,2023-03-04 09:11:00,2023-03-04 10:11:00,2023-03-04 11:11:00,2023-03-04 12:11:00,2023-03-04 13:11:00,2023-03-04 14:11:00,2023-03-04 15:11:00,2023-03-04 16:11:00,2023-03-04 17:11:00,2023-03-04 18:11:00,18:41:00,19:11:00,330,62 days 00:11:00,660
8,大清水局前,40.579968,140.496199,2023-03-04 07:32:00,2023-03-04 08:12:00,2023-03-04 09:12:00,2023-03-04 10:12:00,2023-03-04 11:12:00,2023-03-04 12:12:00,2023-03-04 13:12:00,2023-03-04 14:12:00,2023-03-04 15:12:00,2023-03-04 16:12:00,2023-03-04 17:12:00,2023-03-04 18:12:00,18:42:00,19:12:00,351,62 days 00:12:00,720
9,門外十文字,40.579040,140.499638,2023-03-04 07:33:00,2023-03-04 08:13:00,2023-03-04 09:13:00,2023-03-04 10:13:00,2023-03-04 11:13:00,2023-03-04 12:13:00,2023-03-04 13:13:00,2023-03-04 14:13:00,2023-03-04 15:13:00,2023-03-04 16:13:00,2023-03-04 17:13:00,2023-03-04 18:13:00,18:43:00,19:13:00,383,62 days 00:13:00,780


In [16]:
for i in range(7,18):    
    exec_command2='stops["Stop_'+str(i)+'"]=stops["Stop_'+str(i)+'"].dt.time'
    exec(exec_command2) #すべての時間を時間のみの表示に変更

In [18]:
bus_721=bus[(bus['time'] >= dt.datetime(2021,7,21,17)) & (bus['time'] < dt.datetime(2021,7,21,18))] #各バス停をまとめる
bus_86=bus[(bus['time'] >= dt.datetime(2021,8,6,17)) & (bus['time'] < dt.datetime(2021,8,6,18,6))]
bus_1211=bus[(bus['time'] >= dt.datetime(2021,12,11,9)) & (bus['time'] < dt.datetime(2021,12,11,10,6))]
bus_1211_2=bus[(bus['time'] >= dt.datetime(2021,12,11,10,17)) & (bus['time'] < dt.datetime(2021,12,11,11,25))]
bus_1213=bus[(bus['time'] >= dt.datetime(2021,12,13,8)) & (bus['time'] < dt.datetime(2021,12,13,9,6))]
bus_1214=bus[(bus['time'] >= dt.datetime(2021,12,14,8)) & (bus['time'] < dt.datetime(2021,12,14,9,6))]
bus_1216=bus[(bus['time'] >= dt.datetime(2021,12,16,10)) & (bus['time'] < dt.datetime(2021,12,16,11,6))]
bus_1216_2=bus[(bus['time'] >= dt.datetime(2021,12,16,11,17)) & (bus['time'] < dt.datetime(2021,12,16,12,25))]
bus_1217_2=bus[(bus['time'] >= dt.datetime(2021,12,17,7,17)) & (bus['time'] < dt.datetime(2021,12,17,8,25))]
bus_1223=bus[(bus['time'] >= dt.datetime(2021,12,23,10)) & (bus['time'] < dt.datetime(2021,12,23,11,6))]
bus_1223_2=bus[(bus['time'] >= dt.datetime(2021,12,23,11,17)) & (bus['time'] < dt.datetime(2021,12,23,12,25))]
bus_1231=bus[(bus['time'] >= dt.datetime(2021,12,31,9)) & (bus['time'] < dt.datetime(2021,12,31,10,6))]
bus_1231_2=bus[(bus['time'] >= dt.datetime(2021,12,31,10,17)) & (bus['time'] < dt.datetime(2021,12,31,11,25))]
bus_14=bus[(bus['time'] >= dt.datetime(2022,1,4,13)) & (bus['time'] < dt.datetime(2022,1,4,14,6))]
bus_16=bus[(bus['time'] >= dt.datetime(2022,1,6,10)) & (bus['time'] < dt.datetime(2022,1,6,11,6))]
bus_112=bus[(bus['time'] >= dt.datetime(2022,1,12,9)) & (bus['time'] < dt.datetime(2022,1,12,10,6))]
bus_118=bus[(bus['time'] >= dt.datetime(2022,1,18,10)) & (bus['time'] < dt.datetime(2022,1,18,11,6))]
bus_120=bus[(bus['time'] >= dt.datetime(2022,1,20,10)) & (bus['time'] < dt.datetime(2022,1,20,11,6))]
bus_126=bus[(bus['time'] >= dt.datetime(2022,1,26,10)) & (bus['time'] < dt.datetime(2022,1,26,11,6))]
bus_126_2=bus[(bus['time'] >= dt.datetime(2022,1,26,11,17)) & (bus['time'] < dt.datetime(2022,1,26,12,25))]
bus_128=bus[(bus['time'] >= dt.datetime(2022,1,28,10)) & (bus['time'] < dt.datetime(2022,1,28,11,6))]
bus_128_2=bus[(bus['time'] >= dt.datetime(2022,1,28,11,17)) & (bus['time'] < dt.datetime(2022,1,28,12,25))]
bus_131=bus[(bus['time'] >= dt.datetime(2022,1,31,10)) & (bus['time'] < dt.datetime(2022,1,31,11,6))]
bus_131_2=bus[(bus['time'] >= dt.datetime(2022,1,31,11,17)) & (bus['time'] < dt.datetime(2022,1,31,12,25))]
bus_22=bus[(bus['time'] >= dt.datetime(2022,2,2,10)) & (bus['time'] < dt.datetime(2022,2,2,11,6))]
bus_24=bus[(bus['time'] >= dt.datetime(2022,2,4,10)) & (bus['time'] < dt.datetime(2022,2,4,11,6))]
bus_28=bus[(bus['time'] >= dt.datetime(2022,2,8,11,17)) & (bus['time'] < dt.datetime(2022,2,8,12,25))]
bus_29=bus[(bus['time'] >= dt.datetime(2022,2,9,10)) & (bus['time'] < dt.datetime(2022,2,9,11,6))]
bus_210=bus[(bus['time'] >= dt.datetime(2022,2,10,10)) & (bus['time'] < dt.datetime(2022,2,10,11,6))]
bus_215=bus[(bus['time'] >= dt.datetime(2022,2,15,10)) & (bus['time'] < dt.datetime(2022,2,15,11,6))]
bus_215_2=bus[(bus['time'] >= dt.datetime(2022,2,15,11,17)) & (bus['time'] < dt.datetime(2022,2,15,12,25))]
bus_216=bus[(bus['time'] >= dt.datetime(2022,2,16,10)) & (bus['time'] < dt.datetime(2022,2,16,11,6))]
bus_216_2=bus[(bus['time'] >= dt.datetime(2022,2,16,11,17)) & (bus['time'] < dt.datetime(2022,2,16,12,25))]
bus_217=bus[(bus['time'] >= dt.datetime(2022,2,17,10)) & (bus['time'] < dt.datetime(2022,2,17,11,6))]
bus_217_2=bus[(bus['time'] >= dt.datetime(2022,2,17,11,17)) & (bus['time'] < dt.datetime(2022,2,17,12,25))]
bus_219=bus[(bus['time'] >= dt.datetime(2022,2,19,18)) & (bus['time'] < dt.datetime(2022,2,19,19,6))]
bus_221=bus[(bus['time'] >= dt.datetime(2022,2,21,10)) & (bus['time'] < dt.datetime(2022,2,21,11,6))]
bus_224=bus[(bus['time'] >= dt.datetime(2022,2,24,16)) & (bus['time'] < dt.datetime(2022,2,24,17,6))]
bus_224_2=bus[(bus['time'] >= dt.datetime(2022,2,24,17,17)) & (bus['time'] < dt.datetime(2022,2,24,18,25))]
bus_225=bus[(bus['time'] >= dt.datetime(2022,2,25,10)) & (bus['time'] < dt.datetime(2022,2,25,11,6))]
bus_225_2=bus[(bus['time'] >= dt.datetime(2022,2,25,11,17)) & (bus['time'] < dt.datetime(2022,2,25,12,25))]
bus_33=bus[(bus['time'] >= dt.datetime(2022,3,3,10)) & (bus['time'] < dt.datetime(2022,3,3,11,6))]
bus_37=bus[(bus['time'] >= dt.datetime(2022,3,7,10)) & (bus['time'] < dt.datetime(2022,3,7,11,6))]
bus_39=bus[(bus['time'] >= dt.datetime(2022,3,9,8)) & (bus['time'] < dt.datetime(2022,3,9,9,6))]
bus_313=bus[(bus['time'] >= dt.datetime(2022,3,13,16)) & (bus['time'] < dt.datetime(2022,3,13,17,6))]
bus_313_2=bus[(bus['time'] >= dt.datetime(2022,3,13,17,17)) & (bus['time'] < dt.datetime(2022,3,13,18,25))]
bus_314=bus[(bus['time'] >= dt.datetime(2022,3,14,9)) & (bus['time'] < dt.datetime(2022,3,14,10,6))]
bus_314_2=bus[(bus['time'] >= dt.datetime(2022,3,14,10,17)) & (bus['time'] < dt.datetime(2022,3,14,11,25))]
bus_315=bus[(bus['time'] >= dt.datetime(2022,3,15,9)) & (bus['time'] < dt.datetime(2022,3,15,10,6))]
bus_316_2=bus[(bus['time'] >= dt.datetime(2022,3,16,11,17)) & (bus['time'] < dt.datetime(2022,3,16,12,25))]
bus_318=bus[(bus['time'] >= dt.datetime(2022,3,18,9)) & (bus['time'] < dt.datetime(2022,3,18,10,6))]
bus_318_2=bus[(bus['time'] >= dt.datetime(2022,3,18,11,17)) & (bus['time'] < dt.datetime(2022,3,18,12,25))]
bus_322=bus[(bus['time'] >= dt.datetime(2022,3,22,9)) & (bus['time'] < dt.datetime(2022,3,22,10,6))]
bus_322_2=bus[(bus['time'] >= dt.datetime(2022,3,22,10,17)) & (bus['time'] < dt.datetime(2022,3,22,11,25))]
bus_323=bus[(bus['time'] >= dt.datetime(2022,3,23,9)) & (bus['time'] < dt.datetime(2022,3,23,10,6))]
bus_324=bus[(bus['time'] >= dt.datetime(2022,3,24,9)) & (bus['time'] < dt.datetime(2022,3,24,10,6))]
bus_325=bus[(bus['time'] >= dt.datetime(2022,3,25,9)) & (bus['time'] < dt.datetime(2022,3,25,10,6))]
bus_325_2=bus[(bus['time'] >= dt.datetime(2022,3,25,10,17)) & (bus['time'] < dt.datetime(2022,3,25,11,25))]
bus_328=bus[(bus['time'] >= dt.datetime(2022,3,28,9)) & (bus['time'] < dt.datetime(2022,3,28,10,6))]
bus_331=bus[(bus['time'] >= dt.datetime(2022,3,31,9)) & (bus['time'] < dt.datetime(2022,3,31,10,6))]
bus_331_2=bus[(bus['time'] >= dt.datetime(2022,3,31,10,17)) & (bus['time'] < dt.datetime(2022,3,31,11,25))]
bus_41=bus[(bus['time'] >= dt.datetime(2022,4,1,14)) & (bus['time'] < dt.datetime(2022,4,1,15,6))]
bus_41_2=bus[(bus['time'] >= dt.datetime(2022,4,1,15,17)) & (bus['time'] < dt.datetime(2022,4,1,16,25))]
bus_52=bus[(bus['time'] >= dt.datetime(2022,5,2,9)) & (bus['time'] < dt.datetime(2022,5,2,10,6))]
bus_52_2=bus[(bus['time'] >= dt.datetime(2022,5,2,10,17)) & (bus['time'] < dt.datetime(2022,5,2,11,25))]
bus_52_3=bus[(bus['time'] >= dt.datetime(2022,5,2,16)) & (bus['time'] < dt.datetime(2022,5,2,17,6))]
bus_52_4=bus[(bus['time'] >= dt.datetime(2022,5,2,17,17)) & (bus['time'] < dt.datetime(2022,5,2,18,25))]
bus_524=bus[(bus['time'] >= dt.datetime(2022,5,24,15)) & (bus['time'] < dt.datetime(2022,5,24,16,6))]

In [19]:
route = pd.read_csv('route_oowani.csv')
route=route.drop(route.columns[0],axis=1)
dis_scale = 84.78234392637954
route = route.values.tolist()
def xy2no(xy):
 distances=np.sqrt(np.sum((xy-route)**2,axis=1))*dis_scale
 xy2r_max=np.amin(distances)
 if xy2r_max>0.1:
   no=-1
 else:  
   no=np.argmin(distances)
 return no
I=len(bus)
nos=[]
for i in range(0,I):
 nos.append(xy2no(np.array([bus.iloc[i,1],bus.iloc[i,2]])))
nos_df = pd.DataFrame({'nos':nos})
bus = pd.concat([bus, nos_df], axis=1)
bus = bus[(bus['nos']>=0)]
bus.to_csv("bus_clean_nos.csv")